In [1]:
import waterhealer as wh
import json

In [2]:
source = wh.from_kafka(
    ['testing'],
    {
        'bootstrap.servers': 'localhost:9095, localhost:9096',
        'group.id': 'group3',
        'auto.offset.reset': 'latest',
    })

In [3]:
def json_loads(row):
    return (row[0], json.loads(row[1]))

def plus(row):
    no = row[1]['no']
    if no > 12:
        raise Exception('row is bigger than 12')
    return (row[0], no + 1)

In [4]:
outputs = []
source.map(json_loads).map(plus).partition(5)\
.map(wh.healing_batch, stream = source, callback = print)\
.flatten().sink(outputs.append)

Output()

In [5]:
source.start()

In [6]:
from kafka import KafkaProducer
from kafka.partitioner import RoundRobinPartitioner

In [7]:
producer = KafkaProducer(
    bootstrap_servers = ['localhost:9095', 'localhost:9096'],
    api_version = (0, 10),
    partitioner = RoundRobinPartitioner(),
)

In [8]:
producer.bootstrap_connected()

True

In [9]:
from datetime import datetime
import json
from itertools import cycle

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding = 'utf-8')
        value_bytes = bytes(value, encoding = 'utf-8')
        x = producer_instance.send(topic_name, value = value_bytes)
        return True
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))
        return False

## purposely raise the error

Based on function `plus`, if input bigger than 10, it will throw error.

In [10]:
import confluent_kafka as ck

source.consumer.committed([ck.TopicPartition('testing', 0)])[0].offset

-1001

In [20]:
import time

for count in range(14):
    data = {'no': count, 'datetime': str(datetime.now())}
    publish_message(
        producer, 'testing', 'streaming', json.dumps(data)
    )
    print(source.memory, len(source.memory))
    time.sleep(1.0)

ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 54, 'topic': 'testing'}), ('4-55-testing', {'partition': 4, 'offset': 55, 'topic': 'testing'})]) 2
ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 54, 'topic': 'testing'}), ('4-55-testing', {'partition': 4, 'offset': 55, 'topic': 'testing'})]) 2
ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 54, 'topic': 'testing'}), ('4-55-testing', {'partition': 4, 'offset': 55, 'topic': 'testing'})]) 2
ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 54, 'topic': 'testing'}), ('4-55-testing', {'partition': 4, 'offset': 55, 'topic': 'testing'})]) 2
ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 54, 'topic': 'testing'}), ('4-55-testing', {'partition': 4, 'offset': 55, 'topic': 'testing'})]) 2
ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 54, 'topic': 'testing'}), ('4-55-testing', {'partition': 4, 'offset': 55, 'topic': 'testing'})]) 2
ExpiringDict([('4-54-testing', {'partition': 4, 'offset': 

In [21]:
source.memory

ExpiringDict([('4-54-testing',
               {'partition': 4, 'offset': 54, 'topic': 'testing'}),
              ('4-55-testing',
               {'partition': 4, 'offset': 55, 'topic': 'testing'})])